YOLOv5 model setup and training

In [1]:
import torch
import yaml

In [ ]:
# clone YOLOv5 repo
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!git reset --hard 064365d8683fd002e9ad789c1e91fa3d021b44f0

In [ ]:
# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)

from IPython.display import Image, clear_output  # to display images
from utils.downloads import attempt_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
#download dataset
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="FpynYnn5BBNyvMzeEaoA")
project = rf.workspace("gabijapocevicemajor-7dvfl").project("tattoo-localization-and-recognition")
version = project.version(40)
dataset = version.download("yolov5")

In [ ]:
%cd /content/yolov5

In [ ]:
%cat {dataset.location}/data.yaml

In [7]:
# Define Model Configuration and Architecture
# define number of classes based on YAML
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
#model configuration
%cat /content/yolov5/models/yolov5l.yaml

In [9]:
#customize iPython writefile so I can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [10]:
%%writetemplate /content/yolov5/models/custom_yolov5l.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:
# according to research carried out manually change learning rate at 0.001, momentum at 0.95, the weight decay at 0.0001

In [ ]:
# train yolov5l on tattoo dataset for 300 epochs
%%time
%cd /content/yolov5/
!python train.py --img 640 --batch 64 --epochs 300 --data {dataset.location}/data.yaml --cfg ./models/custom_yolov5l.yaml --weights '' --name yolov5l_results  --cache

Model deployment (to Roboflow)

In [ ]:
project.version(dataset.version).deploy(model_type="yolov5", model_path=f"/content/yolov5/runs/train/yolov5l_results/")

Local picture upload. Bounding box image and cropped image

In [ ]:
!pip install roboflow

In [ ]:
from roboflow import Roboflow
from PIL import Image, ImageDraw

# Initialize Roboflow API with your API key
rf = Roboflow(api_key="FpynYnn5BBNyvMzeEaoA")

# Replace "MODEL_ENDPOINT" with your actual model endpoint
project = rf.workspace().project("tattoo-localization-and-recognition")

# Replace VERSION with the version of your model
model_version = 40
model = project.version(model_version).model

# Replace "your_image.jpg" with the path to your local image
image_path = "/content/1.jpg"

# Infer on a local image
response = model.predict(image_path, confidence=50, overlap=50).json()
predictions = response['predictions']

# Load the original image using PIL
image = Image.open(image_path)

# Specify absolute paths for saving images
output_directory = "/content/"  # Change this to your desired output directory

for bounding_box in predictions:
    x = bounding_box['x']
    y = bounding_box['y']
    width = bounding_box['width']
    height = bounding_box['height']

    # Calculate corner points
    x1 = x - width / 2
    x2 = x + width / 2
    y1 = y - height / 2
    y2 = y + height / 2

    print("Bounding Box Coordinates:")
    print(f"x1: {x1}, y1: {y1}, x2: {x2}, y2: {y2}")

    # Draw bounding box on the image
    draw = ImageDraw.Draw(image)
    draw.rectangle([x1, y1, x2, y2], outline="red", width=3)

    # Crop the bounding box from the original image
    cropped_image = image.crop((x1, y1, x2, y2))

    # Save the cropped image as "cropped.jpg"
    cropped_image.save(output_directory + "cropped.jpg")

# Save the image with bounding box
image.save(output_directory + "prediction.jpg")

Similartity

In [ ]:
pip install pymysql

In [ ]:
from tensorflow.keras.models import Model

In [ ]:
import numpy as np
import requests
from PIL import Image
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from sklearn.metrics.pairwise import cosine_similarity
from io import BytesIO
import pymysql
from tensorflow.keras.models import Model

# Load pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False)
model = Model(inputs=base_model.input, outputs=base_model.layers[-1].output)

# Establish a connection to the MySQL database using public IP
db = pymysql.connect(
    host="34.171.119.255",
    user="root",
    passwd="J44U'QfH}_q*rz>7",  # Enter your password
    db="Identification"
)

def download_image(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content)).resize((224, 224))
    return img

def extract_features(image):
    img_array = np.expand_dims(np.array(image), axis=0)
    img_array = preprocess_input(img_array)  # Preprocess the image for ResNet50
    features = model.predict(img_array)
    return features.flatten()

# Load your cropped image using PIL
cropped_image_path = "/content/cropped.jpg"
cropped_image = Image.open(cropped_image_path).resize((224, 224))  # Resize the image

# Extract features from the cropped image
cropped_features = extract_features(cropped_image)

# Retrieve personal numbers and image URLs from the database
cursor = db.cursor()
cursor.execute("SELECT personal_number, image_url FROM image_table")
personal_info = cursor.fetchall()

# Convert PIL images to numpy arrays and extract features
database_features = []
for personal_number, db_url in personal_info:
    db_image = download_image(db_url).resize((224, 224))  # Resize the image
    db_features = extract_features(db_image)
    database_features.append((personal_number, db_features))

# Compare cropped image features with database features using Cosine similarity
similarity_threshold = 0.45  # Can be adjusted as needed based on experimentation

best_match_index = -1
best_similarity = 0

for i, (personal_number, db_features) in enumerate(database_features):
    similarity = cosine_similarity([cropped_features], [db_features])[0][0]
    if similarity > similarity_threshold and similarity > best_similarity:
        best_match_index = i
        best_similarity = similarity

if best_match_index != -1:
    matched_personal_number = database_features[best_match_index][0]
    cursor.execute("SELECT * FROM personal_info_table WHERE personal_number = %s", matched_personal_number)
    person_data = cursor.fetchone()
    print(f"Match found! Cropped image is most similar to person with personal number: {matched_personal_number}")
    print(f"Similarity score: {best_similarity}")
    print(f"Person data: {person_data}")
else:
    print("No similar image found.")

# Close the database connection
db.close()